In [1]:
# Get the openai secret key:
import getpass

secret_key = getpass.getpass('Please enter your openai key:')

In [6]:
import asyncio
from openai import AsyncOpenAI
import nest_asyncio
from collections import Counter
import json

In [20]:
client = AsyncOpenAI(api_key=secret_key)

nest_asyncio.apply()

async def generate_reasoning_path(question, temperature=0.7):
    """Generates a reasoning path using the OpenAI API asynchronously."""
    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """You are a helpful assistant that provides step-by-step reasoning for questions. Provide a step-by-step reasoning and final answer as keys in JSON step_by_step_reasoning, and final_answer."""},
            {"role": "user", "content": f"{question}"}
        ],
        temperature=temperature,
        max_tokens=1000
    )
    return response.choices[0].message.content

def extract_answer(reasoning_path):
    """Extracts the final answer from a reasoning path."""
    try:
        parsed_json = json.loads(reasoning_path)
        return parsed_json.get('final_answer', '').replace('$', '')
    except json.JSONDecodeError:
        return ""
    
async def self_consistency(question, num_samples=5):
    """Implements the self-consistency method using the OpenAI API asynchronously."""
    tasks = [generate_reasoning_path(question) for _ in range(num_samples)]
    sampled_paths = await asyncio.gather(*tasks)
    answers = [extract_answer(path) for path in sampled_paths]
    most_common_answer = Counter(answers).most_common(1)[0][0]
    return most_common_answer, answers, sampled_paths

# Example usage
async def main():
    question = """Q: There are 5 apples in a basket. If 3 more apples are added, how many apples are in the basket now?
    A: {"step_by_step_reasoning": "There are 5 apples in the basket already. 3 more apples are added. Now there are 5 + 3 = 8 apples. The answer is 8.", "final_answer": "8"}""

    Q: A bakery sells muffins and cakes. Each muffin requires 2 cups of flour and 1 cup of sugar. Each cake requires 3 cups of flour and 2 cups of sugar. The bakery has 200 cups of flour and 150 cups of sugar available. If the profit on each muffin is $5 and on each cake is $8, what is the maximum profit the bakery can make?
    A:"""
    final_answer, answers, reasoning_paths = await self_consistency(question, num_samples=5)

    print(f"The most consistent answer is: {final_answer}")
    print("List of answers:")
    for i, answer in enumerate(answers, 1):
        print(f"Answer {i}: {answer}")
    print("\nReasoning paths:")
    for i, path in enumerate(reasoning_paths, 1):
        print(f"\nPath {i}:")
        print(path)

# Run the async main function
asyncio.run(main())    

The most consistent answer is: 600
List of answers:
Answer 1: 660
Answer 2: 600
Answer 3: 650
Answer 4: 600
Answer 5: 650

Reasoning paths:

Path 1:
{"step_by_step_reasoning": "Let x be the number of muffins and y be the number of cakes. The constraints based on flour and sugar are: 2x + 3y <= 200 (for flour) and x + 2y <= 150 (for sugar). The profit function to maximize is P = 5x + 8y. To find the maximum profit, we can solve the inequalities: 1. From 2x + 3y <= 200, we can express y in terms of x: y <= (200 - 2x)/3. 2. From x + 2y <= 150, we can express y in terms of x: y <= (150 - x)/2. We will graph these inequalities to find the feasible region. The vertices of the feasible region will give us the potential maximum profits. The vertices can be found by solving the equations where the lines intersect: 2x + 3y = 200 and x + 2y = 150. Solving these gives us the points (0, 66.67), (100, 0), and (60, 30). Evaluating the profit function at these points: At (0, 66.67): P = 5(0) + 8(66.67